进球数分析

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
matches = spark.read.format("csv").option("header",True).option("encoding","utf-8").option("sep",",").load("../dataset/WorldCupMatches.csv")
matches.createTempView("goalsGame")
matches.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Datetime: string (nullable = true)
 |-- Stage: string (nullable = true)
 |-- Stadium: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Home Team Name: string (nullable = true)
 |-- Home Team Goals: string (nullable = true)
 |-- Away Team Goals: string (nullable = true)
 |-- Away Team Name: string (nullable = true)
 |-- Attendance: string (nullable = true)
 |-- Half-time Home Goals: string (nullable = true)
 |-- Half-time Away Goals: string (nullable = true)
 |-- Referee: string (nullable = true)
 |-- Assistant 1: string (nullable = true)
 |-- Assistant 2: string (nullable = true)
 |-- RoundID: string (nullable = true)
 |-- MatchID: string (nullable = true)
 |-- Home Team Initials: string (nullable = true)
 |-- Away Team Initials: string (nullable = true)


In [2]:
top10_goals_num_by_country = spark.sql('''
SELECT country, SUM(goals) AS total_goals FROM
((SELECT `Home Team Name` AS country, SUM(CAST(`Home Team Goals` AS INTEGER)) AS goals FROM goalsGame GROUP BY country)
UNION ALL
(SELECT `Away Team Name` AS country, SUM(CAST(`Away Team Goals` AS INTEGER)) AS goals FROM goalsGame GROUP BY country)) t
GROUP BY country Order BY total_goals DESC LIMIT 10
''').show()

+-----------+-----------+
|    country|total_goals|
+-----------+-----------+
|     Brazil|        225|
|  Argentina|        133|
| Germany FR|        131|
|      Italy|        128|
|     France|        108|
|    Germany|        104|
|      Spain|         92|
|Netherlands|         91|
|    Hungary|         87|
|    Uruguay|         80|
+-----------+-----------+


In [3]:
top10_goals_num_by_country_2 = spark.sql('''
SELECT country, home_goals+away_goals AS total_goals,home_goals,away_goals FROM 
(SELECT `Home Team Name` AS country, SUM(CAST(`Home Team Goals` AS INTEGER)) AS home_goals FROM goalsGame GROUP BY country) t1
 JOIN (SELECT `Away Team Name` AS country, SUM(CAST(`Away Team Goals` AS INTEGER)) AS away_goals FROM goalsGame GROUP BY country) t2 
 USING(country)
 ORDER BY total_goals DESC LIMIT 10
''').show()

+-----------+-----------+----------+----------+
|    country|total_goals|home_goals|away_goals|
+-----------+-----------+----------+----------+
|     Brazil|        225|       180|        45|
|  Argentina|        133|       111|        22|
| Germany FR|        131|        99|        32|
|      Italy|        128|        99|        29|
|     France|        108|        68|        40|
|    Germany|        104|        69|        35|
|      Spain|         92|        50|        42|
|Netherlands|         91|        51|        40|
|    Hungary|         87|        73|        14|
|    Uruguay|         80|        62|        18|
+-----------+-----------+----------+----------+
